In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from pyquaternion import Quaternion
import numpy as np
import tqdm

In [3]:
# Loading the nuscene from local system
#nusc = NuScenes(version='v1.0-trainval', dataroot='/home/common_datasets/nuScenes/v1.0-full/', verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot='../data/input_data/v1.0-mini/', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.317 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [5]:
sensor = 'CAM_FRONT'
cam_front_data_token = my_sample['data'][sensor]
cam_front_data = nusc.get('sample_data', cam_front_data_token)
cam_front_data

{'token': 'e3d495d4ac534d54b321f50006683844',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'ego_pose_token': 'e3d495d4ac534d54b321f50006683844',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927612460,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
 'prev': '',
 'next': '68e8e98cf7b0487baa139df808641db7',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [6]:
camera_name = 'CAM_FRONT'

camera_front_bboxes = dict()
# Iterate to scene
for scene in tqdm.tqdm(nusc.scene):
    #Iterate to sample
    token = scene['first_sample_token']
    while token != '':
        sample = nusc.get('sample', token)
        camera_token = sample['data'][camera_name]
        camera_data = nusc.get('sample_data', camera_token)
        annotation_list = sample['anns']

        data_path, boxes, camera_intrinsic = nusc.get_sample_data(camera_token, selected_anntokens=annotation_list)
        if data_path not in camera_front_bboxes:
            camera_front_bboxes[data_path] = []
        for box in boxes:
            corners = view_points(box.corners(), np.array(camera_intrinsic), normalize=True)[:2, :]
            camera_front_bboxes[data_path].append({'name': box.name, 'corners': corners.tolist()})
            
        token = sample['next']
#         print(token)
#         break
        

100%|██████████| 10/10 [00:01<00:00,  5.29it/s]


In [7]:
import json
with open('../data/output_data/camera_front_bboxes.json', 'w') as f:
    json.dump(camera_front_bboxes, f)

In [8]:
camera_front_bboxes[data_path]

[{'name': 'human.pedestrian.adult',
  'corners': [[720.90181671817,
    738.9074819877472,
    738.5828598729023,
    720.6562802615175,
    765.0110268352955,
    782.7587373141831,
    782.2168480972483,
    764.5454183957835],
   [455.96965842373146,
    455.55683254953516,
    567.8784892654493,
    570.1983720519023,
    457.12278831604397,
    456.6785212893316,
    572.1196590488302,
    574.5793317846233]]},
 {'name': 'vehicle.car',
  'corners': [[651.8057152151464,
    464.2641404700817,
    465.81311629883527,
    651.9224347817117,
    486.48901246244264,
    333.65174877639566,
    335.565839818494,
    487.4613285078554],
   [465.72294217223754,
    468.46924320043064,
    652.6226173357721,
    634.1594070819167,
    455.17592729167063,
    456.495304900344,
    595.399718593544,
    584.9466443840229]]},
 {'name': 'human.pedestrian.adult',
  'corners': [[1509.7993690266344,
    1490.6193111435746,
    1488.3473682151564,
    1507.4412919591582,
    1530.3828424023718,
  

In [9]:
len(camera_front_bboxes)

404